### 01_ETL

In [7]:
import os
import fastparquet
import pandas as pd
#from lib import personalised_graphics

#yelp_checkin = pd.read_parquet("data/yelp/checkin.parquet")
#yelp_tip = pd.read_parquet("data/yelp/tip.parquet") 

#### YELP

#### bussines table 1/5

In [24]:
# Import
yelp_business = pd.read_pickle("datos/Yelp/business.pkl")

In [25]:
# Transform
yelp_business = yelp_business.iloc[:, :13] # drop empty columns
yelp_business.drop(columns=["is_open"]) # drop irrelevant data
yelp_business.rename(columns={'stars': 'avg_rating', 'review_count	': 'num_of_reviews'}, inplace=True) # rename columns
columns_order = ["business_id", "name", "address", "city", "state", "latitude", "longitude", "categories", "attributes", "avg_rating"] 
yelp_business = yelp_business.reindex(columns=columns_order) # reorder dataset
yelp_business['categories'] = yelp_business['categories'].astype(str)
yelp_business['categories'] = yelp_business['categories'].str.split(",")

In [ ]:
yelp_business.head()

In [26]:
# Filtrate
# OG ['Hotels & Travel', 'Event Planning & Services', 'Hotels', 'Venues & Event Spaces', 'Bed & Breakfast', 'Home Services', 'Real Estate', 'Apartments', 'Resorts', 'Hostels', 'Vacation Rentals', 'Guest Houses', 'Pet Sitting', 'Pets', 'Property Management', 'Residences', 'Arts & Entertainment', 'Day Spas', 'Landmarks & Historical Buildings', 'Local Services', 'Pet Groomers', 'Public Services & Government', 'Rest Stops', 'Active Life', 'Automotive', 'Commercial Real Estate', 'Parking', 'Pet Boarding', 'Professional Services', 'Retirement Homes', 'Social Clubs']
hotel_bussines = pd.DataFrame()
hotel_categorias = ['Hotels & Travel', 'Hotels', 'Bed & Breakfast', 'Resorts', 'Hostels', 'Vacation Rentals', 'Residences', 'Day Spas', 'Rest Stops']
for categoria in hotel_categorias:
    filtrados = yelp_business[yelp_business['categories'].apply(lambda x: categoria in x)]
    hotel_bussines = pd.concat([hotel_bussines, filtrados], ignore_index=True)
hotel_bussines = hotel_bussines.drop_duplicates(subset=['business_id'])
hotel_bussines.to_csv("Datos/yelp_hotel_business.csv", index=False)

In [27]:
df_hotel_business = pd.DataFrame(hotel_bussines) 

In [28]:
df_hotel_business.shape

(3004, 10)

In [29]:
#Nulls
df_hotel_business.isnull().sum()

business_id      0
name             0
address          0
city             0
state            0
latitude         0
longitude        0
categories       0
attributes     631
avg_rating       0
dtype: int64

In [38]:
df_hotel_business.head(2)

,business_id,name,address,city,state,latitude,longitude,attributes,avg_rating
0,uczmbBk5O3tYhGue13dCDg,new orleans spirit tours,723 st peter st,new orleans,in,29.958431,-90.065173,"{'wifi': ""u'no'""}",4.0
1,Nh1rc9aSeO-Y5lYmXVS6CA,quality inn nashville downtown - stadium,303 interstate dr,nashville,az,36.168834,-86.766911,"{'wifi': ""'free'"", 'businessacceptscreditcards...",2.0


In [ ]:
#df_hotel_business = df_hotel_business.drop('atributos', axis=1)

In [30]:
# haciendo 'attributes' string
df_hotel_business['attributes'] = df_hotel_business['attributes'].astype(str)
df_hotel_business['avg_rating'] = df_hotel_business['avg_rating'].astype(float)
df_hotel_business['name'] = df_hotel_business['name'].astype(str)
df_hotel_business['address'] = df_hotel_business['address'].astype(str)
df_hotel_business['city'] = df_hotel_business['city'].astype(str)

In [31]:
# Drop de 'categories'
categories = df_hotel_business['categories']
df_hotel_business = df_hotel_business.drop('categories', axis=1)

In [37]:
# Specify the column to exclude
column_to_exclude = 'business_id'  # Replace 'column_name' with the actual name of the column to exclude

# Apply lowercase transformation to every column except the specified one
df_hotel_business = df_hotel_business.apply(lambda x: x.str.lower() if x.name != column_to_exclude and pd.api.types.is_string_dtype(x) else x)

In [41]:
df_hotel_business.shape

(3004, 9)

In [83]:
# Por si hay que cambiar el formato de 'attributes'
# df_hotel_business = df_hotel_business.apply(lambda x: x.str.replace('{', ' ') if x.dtype == "object" else x)
# df_hotel_business = df_hotel_business.apply(lambda x: x.str.replace('}', ' ') if x.dtype == "object" else x)
# df_hotel_business = df_hotel_business.apply(lambda x: x.str.replace('"', ' ') if x.dtype == "object" else x)
# df_hotel_business = df_hotel_business.apply(lambda x: x.str.replace("'", " ") if x.dtype == "object" else x)

In [58]:
df_hotel_business.to_csv("Datos/Yelp/Nuevos/yelp_hotel_business.csv", index=False)

#### Yelp reviews table 2/5


###### 3m 15s

In [43]:
import pandas as pd

In [44]:
# import data
yelp_reviews = pd.read_csv("Datos/Yelp/yelp_hotel_reviews.csv")

In [45]:
df_yelp_reviews = pd.DataFrame(yelp_reviews)

In [59]:
df_yelp_reviews.head()

,review_id,user_id,business_id,date,text,stars,useful
0,ra9fNjYUumKp_iaqv-jjDg,l_izt6hn7oTxalU06mSyqg,jTI5Xjk27An8ceJ6VwpXiQ,2013-08-15,"comfortable bed, good breakfast, fast internet...",4.0,0
1,QsuwAldo7cc94nJxu6xKWQ,U0aN9ar9YI24Oc9JshTrDQ,dyCfFIt1XIA8lIpuWaW2eQ,2015-10-08,my parents and i took the oak alley and laura ...,5.0,1
2,O2QGOa66t5BCYDvxLiOtbA,ysU9kNZiFXY7MrFWVIUaTg,cB_9eHHfo9me96cZup-lZQ,2017-04-19,we stayed at the saint last year for a girl's ...,1.0,1
3,Lj1TAURE_hYtPW3IlhKWMg,tWJoKvHynA8h8dFXUExekw,9xm4aIAT_xwRoTxc0_8AWQ,2015-02-16,stayed here three nights on business. the pro...,5.0,0
4,NzvlG7XNjk5cKaCPtZa-eA,3F9mJBT6zQz9CaNo4bAr7g,P61ecBVVPgBm-zH_SYq8FA,2015-08-30,this is one of the best hotels i have ever sta...,5.0,0


In [46]:
df_yelp_reviews.shape

(144162, 7)

In [47]:
# Cambiando tipos de datos
df_yelp_reviews['text'] = df_yelp_reviews['text'].apply(lambda x: str(x).encode('utf-8', 'ignore').decode('utf-8'))

In [48]:
# Minusculas a 'text'
df_yelp_reviews['text'] = df_yelp_reviews['text'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [49]:
# Convert the date column to datetime and keep only date part
df_yelp_reviews['date'] = pd.to_datetime(df_yelp_reviews['date'], format='%Y-%m-%d %H:%M:%S').dt.date

In [56]:
df_yelp_reviews.shape

(144162, 7)

In [51]:
# handle duplicated values
df_yelp_reviews['review_id'] = df_yelp_reviews['review_id'].drop_duplicates()

In [57]:
# Filtrate and export hotel businesses reviews
df_yelp_reviews = df_yelp_reviews[df_yelp_reviews['business_id'].isin(df_hotel_business["business_id"])]
df_yelp_reviews.to_csv("Datos/Yelp/Nuevos/yelp_hotel_reviews.csv", index=False)

#### users table 3/5

In [1]:
import pandas as pd

In [2]:
# import data
yelp_users = pd.read_parquet("Datos/Yelp/user.parquet")

In [4]:
df_yelp_users = pd.DataFrame(yelp_users)

In [20]:
df_yelp_users.head()

,user_id,name,review_count,useful,cool,source_id
0,qVc8ODYU5SZjKXVBgXdI7w,walker,585,7217,5994,0
1,j14WgRoU_-2ZE1aw1dXrJg,daniel,4333,43091,27281,0
2,2WnXYQFK0hXEoTxPtV2zvg,steph,665,2086,1003,0
3,SZDeASXq7o05mMNLshsdIA,gwen,224,512,299,0
4,hA5lMy-EnncsH4JoR-hFGQ,karen,79,29,7,0


In [6]:
# filtrate relevant data
columns_to_keep = ['user_id', 'name', 'review_count', 'useful', 'cool']
df_yelp_users = df_yelp_users[columns_to_keep]

In [11]:
df_yelp_users.shape

(1987897, 5)

In [10]:
# handle duplicated values
duplicados = df_yelp_users[df_yelp_users.duplicated()]
df_yelp_users.drop_duplicates(inplace=True)

In [14]:
# create column to keep track of data source
df_yelp_users['source_id'] = 0

In [19]:
# 'name' a texto para poder pasarlo a minusculas
df_yelp_users['name'] = df_yelp_users['name'].astype(str)
# Minusculas a 'name'
df_yelp_users['name'] = df_yelp_users['name'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [21]:
df_yelp_users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1987897 entries, 0 to 1987896
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   user_id       object
 1   name          object
 2   review_count  int64 
 3   useful        int64 
 4   cool          int64 
 5   source_id     int64 
dtypes: int64(4), object(2)
memory usage: 106.2+ MB


In [59]:
# export data
df_yelp_users.to_csv("Datos/Yelp/Nuevos/yelp_users.csv")
df_yelp_users.to_parquet("Datos/Yelp/Nuevos/yelp_users.parquet", index=False)

#### Yelp_checkin 4/5

In [60]:
import pandas as pd

In [62]:
# import data
yelp_checkin = pd.read_json("Datos/Yelp/checkin.json", lines=True)

In [63]:
df_yelp_checkin = pd.DataFrame(yelp_checkin)

In [64]:
df_yelp_checkin.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


#### Yelp_tip 5/5

In [ ]:
import pandas as pd

# import data
yelp_users = pd.read_parquet("Datos/Yelp/user.parquet")

df_yelp_users = pd.DataFrame(yelp_users)

In [65]:
import pandas as pd

In [78]:
# import data
yelp_tip = pd.read_json("Datos/Yelp/tip.json", lines=True)

In [79]:
# Creando df
df_yelp_tip = pd.DataFrame(yelp_tip)

In [ ]:
# Filtrando a los hoteles
df_yelp_tip = df_yelp_tip[df_yelp_tip['business_id'].isin(df_hotel_business["business_id"])]

In [80]:
df_yelp_tip.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [86]:
df_yelp_tip['compliment_count'].value_counts()

compliment_count
0    13447
1      137
2        9
4        2
3        2
Name: count, dtype: int64

In [85]:
df_yelp_tip.shape

(13597, 5)

In [83]:
# Minusculas a 'text'
df_yelp_tip['text'] = df_yelp_tip['text'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [87]:
df_yelp_tip.to_csv("Datos/Yelp/Nuevos/yelp_tips.csv")

###### 1m 54s